In [2]:
fo_stocks_data = pd.DataFrame()

In [6]:
fo_stocks = ['RELIANCE.NS', 'TATAMOTORS.NS', 'ICICIBANK.NS']  # Replace with the list of F&O stock symbols

for symbol in fo_stocks:
    ticker = yf.Ticker(symbol)
    stock_data = ticker.history(start="2019-01-01", end="2022-01-01")  # Specify the desired date range
    stock_data['Symbol'] = symbol  # Add a column to store the stock symbol
    data = fo_stocks_data.append(stock_data)


C:\Users\annar\AppData\Local\Temp\ipykernel_9868\3529948445.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = fo_stocks_data.append(stock_data)
C:\Users\annar\AppData\Local\Temp\ipykernel_9868\3529948445.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = fo_stocks_data.append(stock_data)
C:\Users\annar\AppData\Local\Temp\ipykernel_9868\3529948445.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = fo_stocks_data.append(stock_data)


In [7]:
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol
Date,,,,,,,,
2019-01-01 00:00:00+05:30,356.552643,359.807469,351.522434,358.771851,7759872,0.0,0.0,ICICIBANK.NS
2019-01-02 00:00:00+05:30,356.552649,362.865055,355.369060,359.610229,18243156,0.0,0.0,ICICIBANK.NS
2019-01-03 00:00:00+05:30,360.004798,362.273309,357.095158,358.278748,12884409,0.0,0.0,ICICIBANK.NS
2019-01-04 00:00:00+05:30,356.897897,363.752765,353.149884,360.202057,22896026,0.0,0.0,ICICIBANK.NS
2019-01-07 00:00:00+05:30,362.470511,365.922611,361.533500,362.667786,12719847,0.0,0.0,ICICIBANK.NS


In [71]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import pyfolio as pf

# Define the start and end dates for data collection
start_date = dt.datetime.now() - dt.timedelta(days=3*365)  # 3 years ago
end_date = dt.datetime.now()

# Get a list of F&O listed stocks in India
fo_stocks = ['TCS.NS', 'RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS', 'TATAMOTORS.NS']  # Example stocks, there are total 198 stocks
l = len(fo_stocks)

# Iterate over each stock
for stock in fo_stocks:
    # Get the historical OHLC data using yfinance
    data = yf.download(stock, start=start_date, end=end_date)

data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-07-01,99.000000,101.449997,98.199997,100.750000,100.750000,50857376
2020-07-02,100.750000,102.949997,100.400002,101.550003,101.550003,48083736
2020-07-03,102.849998,106.349998,102.099998,103.449997,103.449997,87310642
2020-07-06,105.000000,109.900002,104.500000,109.000000,109.000000,91970198
2020-07-07,109.800003,112.599998,108.050003,109.050003,109.050003,85835716


In [72]:
# Calculate the 52-week rolling returns
data['RollingReturns'] = data['Close'].pct_change(52)

# Sort the stocks based on rolling returns and select the top 5 performers
top_performers = data.sort_values('RollingReturns', ascending=False).head(6)

# top_performers = data.groupby('Symbol').tail(1).nlargest(5, 'RollingReturns')

top_performers


,Open,High,Low,Close,Adj Close,Volume,RollingReturns
Date,,,,,,,
2021-01-21,281.000000,299.000000,280.000000,290.600006,290.600006,283614463,1.090648
2021-01-22,295.950012,306.899994,278.350006,289.350006,289.350006,316008609,1.052128
2021-02-02,293.600006,328.850006,290.399994,322.000000,322.000000,251965965,1.037975
2021-01-20,260.000000,277.500000,258.850006,274.899994,274.899994,205145151,0.997094
2021-03-05,333.000000,337.850006,319.700012,325.149994,325.149994,86738909,0.975995


In [73]:
# Define the entry price as the close price of the most recent data point
entry_price = top_performers.iloc[-1]['Close']

# Define the stop loss as 2 times the Average True Range (ATR) on the entry price
atr = top_performers['High'] - top_performers['Low']
stop_loss = entry_price - (2 * atr)

stop_loss

Date
2021-01-21    287.149994
2021-01-22    268.050018
2021-02-02    248.249969
2021-01-20    287.850006
2021-03-05    288.850006
dtype: float64

In [74]:
# Initialize an empty DataFrame to store the strategy performance
strategy_returns = pd.DataFrame()

# Calculate the weekly returns based on rebalancing the portfolio
weekly_returns = abs(top_performers['Close'].pct_change())

# Combine the stock returns with the strategy returns DataFrame
strategy_returns = pd.concat([strategy_returns, weekly_returns], axis=1)

strategy_returns

,Close
2021-01-21 00:00:00,NaN
2021-01-22 00:00:00,0.004301
2021-02-02 00:00:00,0.112839
2021-01-20 00:00:00,0.146273
2021-03-05 00:00:00,0.182794


In [68]:
# Rename the columns of the strategy returns DataFrame with stock symbols
# strategy_returns.set_axis(fo_stocks, axis = 0, inplace = True)

strategy_returns['Stocks'] = fo_stocks

strategy_returns

,Close,Stocks
2020-12-09 00:00:00,NaN,TCS.NS
2020-12-14 00:00:00,0.027957,RELIANCE.NS
2020-12-15 00:00:00,0.014739,HDFCBANK.NS
2020-12-10 00:00:00,0.021426,INFY.NS
2020-12-08 00:00:00,0.002860,ICICIBANK.NS


In [69]:
# Perform the backtest and analyze the strategy using pyfolio
returns = strategy_returns.dropna()

returns

,Close,Stocks
2020-12-14 00:00:00,0.027957,RELIANCE.NS
2020-12-15 00:00:00,0.014739,HDFCBANK.NS
2020-12-10 00:00:00,0.021426,INFY.NS
2020-12-08 00:00:00,0.002860,ICICIBANK.NS


In [70]:
pf.create_full_tear_sheet(returns['Close'])

c:\Users\annar\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
c:\Users\annar\AppData\Local\Programs\Python\Python311\Lib\site-packages\empyrical\stats.py:799: RuntimeWarning: divide by zero encountered in divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
c:\Users\annar\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\plotting.py:648: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


Start date,2020-12-14
End date,2020-12-08
Total months,0
,Backtest
Annual return,6400.5%
Cumulative returns,6.9%
Annual volatility,17.0%
Sharpe ratio,24.81
Calmar ratio,NaN
Stability,0.93
Max drawdown,0.0%


IndexError: index -1 is out of bounds for axis 0 with size 0